## Plot Calexp for pairs for Auxtel

- author Sylvie Dagoret-Campagne
- creation date 2024-06-28
- last update 2024-07-29
- affiliation : IJCLab
- Kernel **w_2024_16**

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import lsst.daf.butler as dafButler
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import pprint
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
#afwDisplay.setDefaultBackend('matplotlib')
afwDisplay.setDefaultBackend('firefly')

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
#collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146' # test this one today 2024/06/22

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
import pandas as pd

## Read the pair file

In [ ]:
pair_num = 93
obj1 = 97
obj2 = 262
file_pair = f"pair_num_{pair_num:.0f}-o{obj1:.0f}-o{obj2:.0f}-colZ-Y.csv"

In [ ]:
file_pair

In [ ]:
df = pd.read_csv(file_pair,index_col=0)

In [ ]:
df[["x11","x12","x21","x22"]]

In [ ]:
df

In [ ]:
count = 0
for idx,row in df.iterrows():
   
    band1 = row["band1"]
    expo1 = row["exposure1"]
    x11 = row["x11"]
    y11 = row["y11"]
    x12 = row["x12"]
    y12 = row["y12"]
    dataId1 = {'visit': expo1, 'instrument':instrument , 'detector': 0}

    calexp1 = butler.get('calexp', **dataId1,collections=collections)
    the_title1 = f"expo {expo1} band {band1}"

    band2 = row["band2"]
    expo2 = row["exposure2"]
    x21 = row["x21"]
    y21 = row["y21"]
    x22 = row["x22"]
    y22 = row["y22"]
    
    dataId2 = {'visit': expo2, 'instrument':instrument , 'detector': 0}
    
    calexp2 = butler.get('calexp', **dataId2,collections=collections)
    the_title2 = f"expo {expo2} band {band2}"

    count+=1
    afw_display = afwDisplay.Display(frame=count)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(calexp1.image,title=the_title1 )
    afw_display.dot('o', x11, y11, size=20, ctype='red')
    afw_display.dot('o', x12, y12, size=20, ctype='blue')
    afw_display.setMaskTransparency(80)

    count+=1
    afw_display = afwDisplay.Display(frame=count)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(calexp2.image,title=the_title2 )
    afw_display.dot('o', x21, y21, size=20, ctype='red')
    afw_display.dot('o', x22, y22, size=20, ctype='blue')
    afw_display.setMaskTransparency(80)

    if idx> 2 :
        break